In [1]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install git+https://github.com/ssut/py-hanspell.git


In [2]:
# !pip install transformers
# !pip install accelerate

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2024-02-11 04:22:42.536300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-11 04:22:43.450375: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-02-11 04:22:43.450474: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-02-11 04:22:43.450485: W tensorfl


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...
Using device: cuda


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


In [4]:
CFG = {
    'LR' : 1e-5, # Learning Rate
    'EPOCHS' : 50, # 학습 Epoch
    'BATCH_SIZE' : 1,
    'AUG_RATIO' : 0.15,
    'AUG_PROB' : 0.5,
}

In [5]:
data_df = pd.read_csv('./data/train.csv')
# test_data = pd.read_csv("./data/test.csv")


In [6]:
import transformers
import re
import random
import numpy as np


class BERT_Augmentation():
    def __init__(self):
        self.model_name = 'monologg/koelectra-base-v3-generator'
        self.model = transformers.AutoModelForMaskedLM.from_pretrained(self.model_name).to(device)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)
        self.unmasker = transformers.pipeline("fill-mask", model=self.model, tokenizer=self.tokenizer)
        random.seed(42)
    def random_masking_replacement(self, sentence, ratio=0.15):
        """Masking random eojeol of the sentence, and recover them using PLM.

        Args:
            sentence (str): Source sentence
            ratio (int): Ratio of masking

        Returns:
          str: Recovered sentence
        """
        
        span = int(round(len(sentence.split()) * ratio))
        
        # 품질 유지를 위해, 문장의 어절 수가 4 이하라면 원문장을 그대로 리턴합니다.
        if len(sentence.split()) <= 4:
            return sentence

        mask = self.tokenizer.mask_token
        unmasker = self.unmasker

        unmask_sentence = sentence
        # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음.
        random_idx = random.randint(1, len(unmask_sentence.split()) - span)
        
        unmask_sentence = unmask_sentence.split()
        # del unmask_sentence[random_idx:random_idx+span]
        cache = []
        for _ in range(span):
            # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음.
            while cache and random_idx in cache:
                random_idx = random.randint(1, len(unmask_sentence) - 2)
            cache.append(random_idx)
            unmask_sentence[random_idx] = mask
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]['sequence']
            unmask_sentence = unmask_sentence.split()
        unmask_sentence = " ".join(unmask_sentence)
        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

    def random_masking_insertion(self, sentence, ratio=0.15):
        
        span = int(round(len(sentence.split()) * ratio))
        mask = self.tokenizer.mask_token
        unmasker = self.unmasker
        
        # Recover
        unmask_sentence = sentence
        
        for _ in range(span):
            unmask_sentence = unmask_sentence.split()
            random_idx = random.randint(0, len(unmask_sentence)-1)
            unmask_sentence.insert(random_idx, mask)
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]['sequence']

        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

In [7]:

# BERT_aug = BERT_Augmentation()

# RATIO=0.15
# for _, row in tqdm(data_df.iterrows()):
#     for q_col in ['질문_1', '질문_2']:
#         # for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
#             # 질문과 답변 쌍을 </s> token으로 연결
#         print(row[q_col])
#         random_masking_replacement = BERT_aug.random_masking_replacement
#         random_masking_insertion = BERT_aug.random_masking_insertion
#         print(random_masking_replacement(row[q_col], CFG['AUG_RATIO']))
#         print(random_masking_insertion(row[q_col], CFG['AUG_RATIO']))
#         print("==")
#         break

In [8]:
import random

# Generate a random boolean with 50% probability
# result = random.choice([True, False])
class HansolDataset(Dataset):
    def __init__(self, data_pd, tokenizer, ratio = CFG['AUG_RATIO']):
        self.tokenizer = tokenizer
        self.data_pd = data_pd
        self.formatted_data = []
        self.sentence_list = []
        self.ratio = ratio
        self._generate_data(data_pd)
        
    def __getitem__(self, index):
        try:
            sentence = self.sentence_list[index]
            question = self._augment_question(sentence["q"]) if random.random() < CFG['AUG_PROB'] else sentence["q"]
            answer = self._augment_question(sentence["a"]) if random.random() < CFG['AUG_PROB'] else sentence["a"]
            input_text = question + tokenizer.eos_token + answer + tokenizer.eos_token
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
        except:
            return self.__getitem__(random.randint(0, len(self.sentence_list) - 1))
        return input_ids
    
    def __len__(self):
        
        return len(self.sentence_list)

    def _augment_question(self, origin_question):
        BERT_aug = BERT_Augmentation()
        random_masking_replacement = BERT_aug.random_masking_replacement
        random_masking_insertion = BERT_aug.random_masking_insertion
        return random_masking_replacement(origin_question, self.ratio) if random.choice([True, False]) else random_masking_insertion(origin_question, self.ratio)

    def _generate_data(self, data):
        for _, row in tqdm(data.iterrows()):
            for q_col in ['질문_1', '질문_2']:
                for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
                    sentence_dict = {}
                    sentence_dict["q"]=row[q_col]
                    sentence_dict["a"]=row[a_col]
                    self.sentence_list.append(sentence_dict)
                    


In [9]:
# tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>')
tokenizer = PreTrainedTokenizerFast.from_pretrained('./skt_5_2_epoch/', eos_token='</s>')
model = GPT2LMHeadModel.from_pretrained("./skt_5_2_epoch/")

In [10]:

dataset = HansolDataset(data_pd = data_df,tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True) # 미니 배치 형태로 데이터 갖추기

644it [00:00, 8780.14it/s]


In [11]:
# for name, module in model.named_children():
#     if name == "gpt_neox":
#         for name, sub_module in module.named_children():
#             # print(name)
#             if name == "embed_in":
#                 for param in sub_module.parameters():
#                     param.requires_grad = False
#             elif name == "emb_dropout":
#                 for param in sub_module.parameters():
#                     param.requires_grad = False
#             elif name == "layers":
#                 # print(name)
#                  for name, layer_module in sub_module.named_children():
#                      if int(name) < 26:
#                         for param in layer_module.parameters():
#                             param.requires_grad = False
            

In [12]:

model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    # for batch in tqdm(dataloader, desc="Processing batches", unit="batch"):

    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        # loss_2 = LossFunction(outputs,batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(dataloader)}")
    model.save_pretrained(f"./skt_augment_{epoch}_epoch")
    tokenizer.save_pretrained(f"./skt_augment_{epoch}_epoch")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 0.6276:  55%|██████████████████████████████████████████████████████████████████████████▊                                                            | 3568/6440 [2:01:47<1:21:32,  1.70s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 2 - Avg Loss: 0.3856:  51%|████████████████████████████████████████████████████████████████████▉                                                                

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   │   optimizer.step()                                                                    │
│   21 │   │   optimizer.zero_grad()                                                               │
│   22 │   │                                                                                       │
│ ❱ 23 │   │   total_loss += loss.item()                                                           │
│   24 │   │                                                                                       │
│   25 │   │   # 진행률 표시줄에 평균 손실 업데이트                                                │
│   26 │   │   progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
ㅣ